In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from scipy.io import loadmat, savemat

from tensorflow.keras import layers

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [3]:
Y_4802 = loadmat('Y_4802.mat')['Y_4802']

In [4]:
sequence = loadmat('dataset_4802.mat')['Sequence']

In [5]:
amino_code = {'A':0, 'C':1, 'D':2, 'E':3, 'F':4, 'G':5, 'H':6,
             'I':7, 'K':8, 'L':9, 'M':10, 'N':11, 'P':12, 
             'Q':13, 'R':14, 'S':15, 'T':16, 'U':17, 'V':18,
             'W':19, 'X':20, 'Y':21 }

In [6]:
extend_one_encode = []
seq_num_encode = []
for s in sequence:
    p_seq = (list(s[0][0]) * 1000)[0:10000]
    seq_num = np.array([amino_code[x] for x in p_seq])
    seq_num_encode.append(seq_num)
    tmp = np.zeros((seq_num.size, 22))
    tmp[np.arange(seq_num.size), seq_num] = 1
    extend_one_encode.append(tmp)

In [7]:
X_4802_padding = np.array(extend_one_encode)

In [8]:
X_4802_padding.shape

(4802, 10000, 22)

In [9]:
X_4802_num = np.array(seq_num_encode); X_4802_num.shape

(4802, 10000)

In [10]:
X_4802_padding[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
Y_4802.shape

(4802, 37)

In [13]:
#inputs = keras.Input(shape=(None, 22,), dtype = "float32")
inputs = keras.Input(shape=(None,), dtype = "int32")
x = layers.Embedding(22, 22)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
#x = layers.Bidirectional(layers.LSTM(256, return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(128))(x)
x = layers.Reshape((16,16,1))(x)

x = layers.Conv2D(256, (4,3), activation='relu')(x)
x = layers.Conv2D(128, 3, activation='relu',)(x)
# x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',)(x)
# x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3, activation='relu',)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(37, activation='sigmoid')(x)

# x = layers.Conv2D(128, 3, activation='relu',)(x)
# x = layers.Conv2D(128, 3, activation='relu',)(x)
# x = layers.Conv2D(64, 3, activation='relu',)(x)
# x = layers.Conv2D(32, 3, activation='relu',)(x)
# x = layers.Flatten()(x)
# outputs = layers.Dense(37, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 22)          484       
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 128)         44544     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               263168    
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 14, 256)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 12, 128)      

In [14]:
#tensorboard_callback = keras.callbacks.TensorBoard()

model.compile("adam", "binary_crossentropy", metrics=["binary_accuracy"])
model.fit(X_4802_num, Y_4802, batch_size=32, epochs=2,)
         # callbacks = [tensorboard_callback])#, validation_data=(x_val, y_val))

Epoch 1/2
151/151 [==============================] - 268s 2s/step - loss: 0.1396 - accuracy: 0.1874
Epoch 2/2
151/151 [==============================] - 274s 2s/step - loss: 0.1058 - accuracy: 0.1999


In [ ]:
X_4802_feature = loadmat('feature_4802.mat')

In [ ]:
pssmpse = X_4802_feature['pssmpse']

In [ ]:
pssmpse[0].shape

In [ ]:
X_4802_feature.keys()

In [ ]:
pppse = X_4802_feature['pppse']

In [ ]:
pppse.shape

In [ ]:
input_shape = (4, 7, 28, 28, 3)
input_shape[2:]

In [ ]:
input_shape = pssmpse.shape
pssm = keras.Input(shape=input_shape[1:], dtype = "float32")

x = layers.Conv2D(256, (4,3), activation='relu')(pssm)
x = layers.Conv2D(128, 3, activation='relu',)(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',)(x)
#x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3, activation='relu',)(x)
outputs = layers.Flatten()(x)
#outputs = layers.Dense(37, activation='sigmoid')(x)
model_pssm = keras.Model(pssm, outputs)
model_pssm.summary()

In [ ]:
input_shape = pppse.shape
pse = keras.Input(shape=input_shape[1:], dtype = "float32")

x = layers.Conv2D(256, (4,3), activation='relu')(pse)
x = layers.Conv2D(128, 3, activation='relu',)(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',)(x)
#x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3, activation='relu',)(x)
outputs = layers.Flatten()(x)
#outputs = layers.Dense(37, activation='sigmoid')(x)
model_pse = keras.Model(pse, outputs)
model_pse.summary()

In [ ]:
tmodel = layers.Concatenate()([model_pssm.output, model_pse.output])
x = layers.Dense(37, activation='sigmoid')(tmodel)
model = keras.Model(inputs=[pssm, pse], outputs = x)
model.summary()

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit([pssmpse, pppse], Y_4802, batch_size=8, epochs=50,validation_split=0.1)
         # callbacks = [tensorboard_callback])#, validation_data=(x_val, y_val))

In [ ]:
res = model.predict([pssmpse[0:10], pppse[0:10]])

In [ ]:
max(res.flatten())

In [ ]:
min(res.flatten())

In [ ]:
res[res>=0.5] = 1
res[res<0.5] = 0

In [ ]:
res